In [98]:
import tcxparser
import skmob
import pandas as pd
import os
from functools import partial
import pyproj
from shapely.ops import transform
import geojson
from shapely.geometry import Point
import json
from shapely.geometry import shape


In [2]:
def trace_parser(filename):
    try:
        trace = tcxparser.TCXParser(filename)
        position_n = len(trace.position_values())
        trace_df = pd.DataFrame(data={"position_values":trace.position_values()[:position_n],
                                      "altitude_points":trace.altitude_points()[:position_n],
                                      "time_values": trace.time_values()[:position_n], 
                                      "hr_values": trace.hr_values()[:position_n] 
                                     })

        trace_df["lat"] = trace_df.position_values.map(lambda x: x[0])
        trace_df["lon"] = trace_df.position_values.map(lambda x: x[1])
        trace_df["started_at"] = trace.started_at
        trace_df["time_values"] = pd.to_datetime(trace_df.time_values)
        
        filename = filename.split("_")[1].split(".")[0]
        trace_df["id"] = filename
        return trace_df
    except Exception as e:
        print(e)
        print(filename)


In [11]:
DATA_DIR = "../../raw_data"

In [12]:
trace = trace_parser(f"{DATA_DIR}/running_canal_ourcq.tcx")
tdf= skmob.TrajDataFrame(trace, latitude="lat", longitude="lon", datetime="time_values", user_id='id')
tdf = tdf[0:550]

tdf[0:550].plot_trajectory(zoom=12, weight=3, opacity=0.9, tiles='cartodbpositron')

In [13]:
project = partial(
    pyproj.transform,
    pyproj.Proj(init='epsg:4326'), # source coordinate system
    pyproj.Proj(init='epsg:2154')) # destination coordinate system

tdf.position_values = tdf.position_values.map(lambda x: Point(x))
tdf.position_values = tdf.position_values.map(lambda x: transform(project, x))
tdf[0:250].to_csv(f"{DATA_DIR}/trace.csv", index = False)


In [99]:
sea_df = pd.read_csv(f"{DATA_DIR}/europe-sea.csv", encoding = 'utf-8', sep = ';')
sea_df = sea_df[sea_df["Geo Shape"].notnull()]
sea_df["the_geom"] = sea_df["Geo Shape"].map(lambda x: transform(project, shape(json.loads(x))))

In [100]:
sea_df.columns = [col.lower().replace(" ", "_") for col in sea_df.columns]
sea_df.to_csv(f"{DATA_DIR}/europe-sea-prep.csv", index = False)

In [101]:
sea_df.columns

Index(['geo_point', 'geo_shape', 'fcsubtype', 'inspire_id', 'begin_lifes',
       'f_code', 'icc', 'shape__leng', 'shape__area', 'the_geom'],
      dtype='object')